In [7]:
from pathlib import Path
from sqlalchemy import create_engine

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

# Se define la ruta para la base de datos local SQLite
project_name = Path.cwd().parent.parent.name
data_projetc_path = Path.home().joinpath(f'Dropbox/La Casa Del Carpintero/Departamento de Programación/data_projects_git/data_{project_name}')
db_file = 'comisiones.db'
db_file_path_str = str(data_projetc_path.joinpath(db_file))

# Se crea el objeto engine para trabajarlo con los metodos de pandas
engine = create_engine(f'sqlite:///{db_file_path_str}')

In [8]:
file01 = 'clientes_regalos'
file01_path_str = str(data_projetc_path.joinpath(f'comisiones/{file01}.xlsx'))
clientes_regalos = pd.read_excel(file01_path_str, dtype_backend='numpy_nullable')

file02 = 'refacturaciones'
file02_path_str = str(data_projetc_path.joinpath(f'costo_ventas/{file02}.xlsx'))
refacturaciones = pd.read_excel(file02_path_str, dtype_backend='numpy_nullable')

In [11]:
with engine.connect() as conn, conn.begin():

    cobranza_enero = pd.read_sql_table('cobranza_1_2024', conn, dtype_backend='numpy_nullable')
    cobranza_febrero = pd.read_sql_table('cobranza_2_2024', conn, dtype_backend='numpy_nullable')
    cobranza_marzo = pd.read_sql_table('cobranza_3_2024', conn, dtype_backend='numpy_nullable')
    cobranza_abril = pd.read_sql_table('cobranza_4_2024', conn, dtype_backend='numpy_nullable')
    cobranza_mayo = pd.read_sql_table('cobranza_5_2024', conn, dtype_backend='numpy_nullable')
    cobranza_junio = pd.read_sql_table('cobranza_6_2024', conn, dtype_backend='numpy_nullable')
    cobranza_julio = pd.read_sql_table('cobranza_7_2024', conn, dtype_backend='numpy_nullable')
    cobranza_año = pd.read_sql_table('cobranza_año_2024', conn, dtype_backend='numpy_nullable')

    costo_ventas_enero = pd.read_sql_table('costo_ventas_1_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_febrero = pd.read_sql_table('costo_ventas_2_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_marzo = pd.read_sql_table('costo_ventas_3_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_abril = pd.read_sql_table('costo_ventas_4_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_mayo = pd.read_sql_table('costo_ventas_5_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_junio = pd.read_sql_table('costo_ventas_6_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_julio = pd.read_sql_table('costo_ventas_7_2024', conn, dtype_backend='numpy_nullable')
    
    datos_com_2023 = pd.read_sql_table('datos_com_2023', conn, dtype_backend='numpy_nullable')
    bonos_limites_comisiones = pd.read_sql_table('bonos_limites_comisiones', conn, dtype_backend='numpy_nullable')

engine.dispose()

costo_ventas_año = pd.concat([costo_ventas_enero, costo_ventas_febrero, costo_ventas_marzo, costo_ventas_abril, costo_ventas_mayo, costo_ventas_junio, costo_ventas_julio])

In [12]:
costo_ventas_año['mes'] = costo_ventas_año['invoice_date'].dt.month

(costo_ventas_año.loc[
        (costo_ventas_año['salesperson_id'] == 211)
        ,[
            'mes',
            'utilidad_partida_$_after_cc'
        ]
    ]
    .groupby('mes')
    .sum()
)

,utilidad_partida_$_after_cc
mes,
1,"587,982.60"
2,"270,236.17"
3,"545,186.95"
4,"260,344.33"
5,"253,872.19"
6,"238,872.74"
7,"184,086.56"


In [65]:
cobranza = cobranza_marzo
wep = cobranza.loc[
            (cobranza['paga_comision'] == True)
            , 
            ['fact_doc_name', 'fact_doc_date', 'pay_date', 'dias_pago']
        ]

wep['año'] = wep['fact_doc_date'].dt.year
wep['mes'] = wep['fact_doc_date'].dt.month

wep.groupby(['año', 'mes']).count()

fact_doc_name  fact_doc_date  pay_date  dias_pago
año  mes                                                   
2022 11               1              1         1          1
2023 9                1              1         1          1
     10              23             23        23         23
     12              32             32        32         32
2024 1              111            111       111        111
     2              402            402       402        402
     3             3479           3479      3479       3479
     4                3              3         3          3
     6                1              1         1          1

In [31]:
datos_com_2023

,salesperson_id,salesperson_name,V11-2023,C11-2023,V12-2023,C12-2023
0,205,Brenda Luz Acosta Lopez,"1,200,567.44","12,336.75","756,619.80","8,962.43"
1,218,Gladiz Melisa Galvez Espinoza,"346,733.12","3,283.35","540,410.57","4,859.61"
2,210,Irma Carvajal Flores,"1,561,072.02","16,723.57","845,081.98","10,101.28"
3,221,Karla Jaqueline Rivera Hernández,"962,898.57","8,043.62","670,175.57","5,349.91"
4,224,Leticia Terán Salinas,"248,271.55","1,178.20","202,762.89",922.10
5,206,Mariana Araceli Carvajal Flores,"1,603,457.64","7,086.81","1,529,760.13","6,942.88"
6,211,Mayra Angelica Parada Manjarrez,"1,412,102.93","17,744.85","1,114,001.03","16,281.31"
7,216,Nadia Santos Nava,"607,513.73","4,916.97","450,516.61","4,144.39"
8,227,Patricia Flores Pantaleón,"231,786.79","2,526.84","229,339.68","2,583.58"
9,212,Reyna Arriaga Flores,"735,239.37","5,314.66","437,679.98","3,811.39"
